In [1]:
!pip install pandas==1.3.4
!pip install transformers==4.12.5
!pip install datasets==1.15.1
!pip install ipywidgets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 133 kB 17.3 MB/s eta 0:00:01
     |████████████████████████████████| 2.0 MB 40.8 MB/s eta 0:00:01
     |████████████████████████████████| 383 kB 65.6 MB/s eta 0:00:01


In [2]:
import os
import pickle

from collections import Counter

# import pandas as pd
from sklearn.metrics import classification_report

import numpy as np
import torch
import torch.nn as nn

import transformers
from transformers import Trainer
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers.data.data_collator import DataCollatorWithPadding

import datasets
from datasets import Dataset
from datasets import ClassLabel
from datasets import load_metric

## Global variables

In [3]:
DATA_FOLDER = '/notebooks/Data/bert_sequence_classification'
DATA_FILE = '/notebooks/Prompting/notebooks/link_task_prompting/pe_dataset_w_prompt_rep_2.pt'
RESULTS_FOLDER = '/notebooks/Results/bert_sequence_classification'

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
device

device(type='cuda')

## Load data

In [6]:
dataset = torch.load(DATA_FILE)

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['essay_nr', 'component_id', 'label_and_comp_idxs', 'text', 'label_x', 'label_ComponentType', 'relation_SupportAttack', 'label_RelationType', 'labels', 'split', 'essay', 'argument_bound_1', 'argument_bound_2', 'argument_id', 'sentence', 'paragraph', 'para_nr', 'total_paras', 'token_count', 'token_count_covering_para', 'tokens_count_covering_sentence', 'preceeding_tokens_in_sentence_count', 'succeeding_tokens_in_sentence_count', 'token_ratio', 'relative_position_in_para_char', 'is_in_intro', 'relative_position_in_para_token', 'is_in_conclusion', 'is_first_in_para', 'is_last_in_para', 'nr_preceeding_comps_in_para', 'nr_following_comps_in_para', 'structural_fts_as_text', 'structural_fts_as_text_combined', 'para_ratio', 'first_or_last', 'strct_fts_w_position_in_essay', 'component_pos_tags', 'strct_fts_essay_position_pos_tags', 'prompted_representation_2'],
        num_rows: 3770
    })
    test: Dataset({
        features: ['essay_nr', '

In [8]:
dataset['train']['strct_fts_essay_position_pos_tags'][230]

"Topic: Young people should go to university or not, Sentence: Although higher education does not guarantee young students' success, the benefits of learning in universities are the vital factor in creating more possibilities for the development of society through advancing academic fulfillment and the young generation who have experiences and challenges., First or last in essay: Yes, First in paragraph: No, Last in paragraph: Yes, In in introduction: No, Is in conclusion: Yes. Part Of Speech tags: DET, NOUN, ADP, VERB, ADP, NOUN, AUX, DET, ADJ, NOUN, ADP, VERB, ADJ, NOUN, ADP, DET, NOUN, ADP, NOUN, ADP, VERB, ADJ, NOUN, CCONJ, DET, ADJ, NOUN, PRON, AUX, NOUN, CCONJ, NOUN"

In [9]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [10]:
label_names = set(dataset['train']['labels'])
label_nb = len(label_names)
labels = ClassLabel(num_classes=label_nb, names=label_names)

In [11]:
# label_names = set(['Separate', 'Linked'])
# label_nb = len(label_names)
# labels = ClassLabel(num_classes=label_nb, names=label_names)

In [12]:
labels

ClassLabel(num_classes=2, names={'Separate', 'Linked'}, names_file=None, id=None)

In [13]:
def tokenize(batch):
    tokens = tokenizer(batch['strct_fts_essay_position_pos_tags'], truncation=True, padding=True, max_length=512)
    tokens['labels'] = labels.str2int(batch['labels'])
    return tokens

# this is just the text. if the results are nice, check transfer with text + topic 

In [14]:
dataset = dataset.map(tokenize, batched=True)

Parameter 'function'=<function tokenize at 0x7f2e83033ee0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [15]:
dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

In [16]:
dataset

DatasetDict({
    train: Dataset({
        features: ['argument_bound_1', 'argument_bound_2', 'argument_id', 'attention_mask', 'component_id', 'component_pos_tags', 'essay', 'essay_nr', 'first_or_last', 'input_ids', 'is_first_in_para', 'is_in_conclusion', 'is_in_intro', 'is_last_in_para', 'label_ComponentType', 'label_RelationType', 'label_and_comp_idxs', 'label_x', 'labels', 'nr_following_comps_in_para', 'nr_preceeding_comps_in_para', 'para_nr', 'para_ratio', 'paragraph', 'preceeding_tokens_in_sentence_count', 'prompted_representation_2', 'relation_SupportAttack', 'relative_position_in_para_char', 'relative_position_in_para_token', 'sentence', 'split', 'strct_fts_essay_position_pos_tags', 'strct_fts_w_position_in_essay', 'structural_fts_as_text', 'structural_fts_as_text_combined', 'succeeding_tokens_in_sentence_count', 'text', 'token_count', 'token_count_covering_para', 'token_ratio', 'token_type_ids', 'tokens_count_covering_sentence', 'total_paras'],
        num_rows: 3770
    })
   

In [17]:
train_dataset = dataset['train'].shuffle(seed=42)
test_dataset = dataset['test'].shuffle(seed=42)
val_dataset = dataset['validation'].shuffle(seed=42)

# train_val_datasets = dataset['train'].train_test_split(train_size=0.8, seed=42)
# train_dataset = train_val_datasets['train']
# val_dataset = train_val_datasets['test']

In [18]:
dataset_d = {}
dataset_d['train'] = train_dataset
dataset_d['test'] = test_dataset
dataset_d['val'] = val_dataset

In [19]:
tokenizer.decode(dataset['train'][2945]['input_ids'])

'[CLS] topic : zoos have no useful purpose?, sentence : in the zoo you can see an animal and their different variations, the male and the female or the baby and the adult, first or last in essay : no, first in paragraph : no, last in paragraph : no, in in introduction : no, is in conclusion : no. part of speech tags : adp, det, noun, pron, verb, verb, det, noun, cconj, det, adj, noun, punct, det, noun, cconj, det, noun, cconj, det, noun, cconj, det, noun [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

In [20]:
# sanity check
set(dataset_d['train']['split'])

{'TRAIN'}

In [21]:
# sanity check
set(dataset_d['val']['split'])

{'TRAIN'}

In [22]:
# sanity check
set(dataset_d['test']['split'])

{'TEST'}

In [23]:
# global variables
NUM_LABELS = labels.num_classes
BATCH_SIZE = 48
NB_EPOCHS = 6

In [24]:
model = BertForSequenceClassification.from_pretrained("/notebooks/Prompting/notebooks/link_task_prompting/pe_mask_model_prompted_rep_2_linkedID", num_labels=NUM_LABELS)
model.to(device)

Some weights of the model checkpoint at /notebooks/Prompting/notebooks/link_task_prompting/pe_mask_model_prompted_rep_2_linkedID were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were n

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [25]:
# https://huggingface.co/transformers/main_classes/trainer.html
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get('logits')
        loss_fct = nn.CrossEntropyLoss()#(weight=class_weights)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [26]:
metric = load_metric('f1')

def compute_metrics(eval_pred):
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    return metric.compute(predictions=predictions, references=labels, average='macro')

Downloading:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

In [27]:
training_args = TrainingArguments(
    
    # output
    output_dir=RESULTS_FOLDER,          
    
    # params
    num_train_epochs=NB_EPOCHS,               # nb of epochs
    per_device_train_batch_size=BATCH_SIZE,   # batch size per device during training
    per_device_eval_batch_size=BATCH_SIZE,    # cf. paper Sun et al.
    learning_rate=1e-5,#2e-5,                 # cf. paper Sun et al.
#     warmup_steps=500,                         # number of warmup steps for learning rate scheduler
    warmup_ratio=0.1,                         # cf. paper Sun et al.
    weight_decay=0.01,                        # strength of weight decay
    
    # eval
    evaluation_strategy="steps",              # cf. paper Sun et al.
    eval_steps=20,                            # cf. paper Sun et al.
    
    # log
    logging_dir="/notebooks/Results/bert_sequence_classification/tb_logs",  
    logging_strategy='steps',
    logging_steps=20,
    
    # save
    save_strategy='steps',
    save_total_limit=2,
    # save_steps=20, # default 500
    load_best_model_at_end=True,              # cf. paper Sun et al.
    # metric_for_best_model='eval_loss' 
    metric_for_best_model='f1'
)

In [28]:
trainer = CustomTrainer( # Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
    # callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

In [29]:
results = trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: label_x, text, sentence, relation_SupportAttack, relative_position_in_para_char, strct_fts_w_position_in_essay, structural_fts_as_text, structural_fts_as_text_combined, token_ratio, label_RelationType, argument_bound_1, token_count_covering_para, para_nr, tokens_count_covering_sentence, paragraph, relative_position_in_para_token, succeeding_tokens_in_sentence_count, is_in_intro, nr_following_comps_in_para, nr_preceeding_comps_in_para, strct_fts_essay_position_pos_tags, label_and_comp_idxs, essay_nr, preceeding_tokens_in_sentence_count, essay, is_in_conclusion, first_or_last, component_pos_tags, argument_bound_2, is_first_in_para, prompted_representation_2, split, argument_id, para_ratio, label_ComponentType, total_paras, component_id, is_last_in_para, token_count.
***** Running training *****
  Num examples = 3770
  Num Epochs = 6
  Instantane

Step,Training Loss,Validation Loss,F1
20,0.673700,0.670677,0.372588
40,0.660700,0.649126,0.383028
60,0.632000,0.581038,0.681994
80,0.574000,0.535819,0.716216
100,0.527900,0.553929,0.724944
120,0.492600,0.573463,0.732946
140,0.498500,0.529736,0.738180
160,0.522500,0.515503,0.736260
180,0.486600,0.560767,0.736199
200,0.471200,0.548577,0.731683


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: label_x, text, sentence, relation_SupportAttack, relative_position_in_para_char, strct_fts_w_position_in_essay, structural_fts_as_text, structural_fts_as_text_combined, token_ratio, label_RelationType, argument_bound_1, token_count_covering_para, para_nr, tokens_count_covering_sentence, paragraph, relative_position_in_para_token, succeeding_tokens_in_sentence_count, is_in_intro, nr_following_comps_in_para, nr_preceeding_comps_in_para, strct_fts_essay_position_pos_tags, label_and_comp_idxs, essay_nr, preceeding_tokens_in_sentence_count, essay, is_in_conclusion, first_or_last, component_pos_tags, argument_bound_2, is_first_in_para, prompted_representation_2, split, argument_id, para_ratio, label_ComponentType, total_paras, component_id, is_last_in_para, token_count.
***** Running Evaluation *****
  Num examples = 943
  Batch size = 48
The foll

In [30]:
# save best model
# trainer.save_model(os.path.join("/notebooks/Results/bert_sequence_classification", 'best-model-for-probs'))

In [31]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [32]:
test_trainer = Trainer(model, data_collator=DataCollatorWithPadding(tokenizer))
test_raw_preds, test_labels, _ = test_trainer.predict(test_dataset)
test_preds = np.argmax(test_raw_preds, axis=1)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: label_x, text, sentence, relation_SupportAttack, relative_position_in_para_char, strct_fts_w_position_in_essay, structural_fts_as_text, structural_fts_as_text_combined, token_ratio, label_RelationType, argument_bound_1, token_count_covering_para, para_nr, tokens_count_covering_sentence, paragraph, relative_position_in_para_token, succeeding_tokens_in_sentence_count, is_in_intro, nr_following_comps_in_para, nr_preceeding_comps_in_para, strct_fts_essay_position_pos_tags, label_and_comp_idxs, essay

In [33]:
len(test_preds)

1260

In [34]:
target_name = labels.int2str([0,1])
print(classification_report(test_labels, test_preds, target_names=target_name, digits=3))

              precision    recall  f1-score   support

    Separate      0.829     0.782     0.805       744
      Linked      0.710     0.767     0.737       516

    accuracy                          0.776      1260
   macro avg      0.769     0.775     0.771      1260
weighted avg      0.780     0.776     0.777      1260

